In [1]:
# stdlib
import os
from pathlib import Path

# syft absolute
import syft as sy

In [2]:
server = sy.orchestra.launch(
    name="test_upgradability",
    dev_mode=True,
    reset=False,
    port="auto",
    migrate=False,
)

client = server.login(email="info@openmined.org", password="changethis")

Autoreload enabled
Starting test_upgradability server on 0.0.0.0:61718


INFO:     Will watch for changes in these directories: ['/Users/koen/workspace/PySyft/packages/syft/src/syft']
INFO:     Uvicorn running on http://0.0.0.0:61718 (Press CTRL+C to quit)
INFO:     Started reloader process [42910] using WatchFiles


Waiting for server to start

INFO:     Started server process [42914]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


 Done.


SyftInfo: You have launched a development server at http://0.0.0.0:61718.It is intended only for local use.

Logged into <test_upgradability: High side Datasite> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].account.set_password([new_password])`.

In [3]:
# client = sy.login(email="info@openmined.org", password="changethis", port=8080)

In [4]:
# Check if this server has data on it
assert len(client.users.get_all()) == 2

In [5]:
migration_data = client.get_migration_data(include_blobs=True)
migration_data

```python
class MigrationData:
  id: str = 96822432841c4a04b7aca48e48907160
  server_uid: str = 21519e1e3e664b38a635dc951c293158
  root_verify_key: str = 119468ba6f6bd124351b4cbce80b12fdf0c443294151fd3ece3833eff825ad1e
  num_objects: str = 30
  num_action_objects: str = 2
  includes_blobs: str = True

```

In [6]:
assert migration_data.includes_blobs
assert migration_data.num_action_objects > 0

In [7]:
migration_data_dir = Path(os.getenv("MIGRATION_DATA_DIR", "."))
migration_data_dir.mkdir(exist_ok=True)

blob_path = migration_data_dir / "migration.blob"
yaml_path = migration_data_dir / "migration.yaml"

blob_path.unlink(missing_ok=True)
yaml_path.unlink(missing_ok=True)

In [8]:
migration_data.save(blob_path, yaml_path=yaml_path)

assert blob_path.exists()
assert yaml_path.exists()

print(f"Saved migration data to {str(blob_path.resolve())}")

Saved migration data to /Users/koen/workspace/PySyft/notebooks/tutorials/version-upgrades/migration.blob


In [9]:
if server.server_type.value == "python":
    server.land()